In [1]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
PROJECT_NUM = !gcloud projects list --filter="$PROJECT_ID" --format="value(PROJECT_NUMBER)"
PROJECT_NUM = PROJECT_NUM[0]
LOCATION = 'us-central1'
REGION = "us-central1"

# VERTEX_SA = '934903580331-compute@developer.gserviceaccount.com'
VERTEX_SA = 'jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com'

print(f"PROJECT_ID: {PROJECT_ID}")
print(f"PROJECT_NUM: {PROJECT_NUM}")
print(f"LOCATION: {LOCATION}")
print(f"REGION: {REGION}")
print(f"VERTEX_SA: {VERTEX_SA}")

PROJECT_ID: hybrid-vertex
PROJECT_NUM: 934903580331
LOCATION: us-central1
REGION: us-central1
VERTEX_SA: jt-vertex-sa@hybrid-vertex.iam.gserviceaccount.com


In [2]:
REGION = 'us-central1'
EXPERIMENT = 'control_group1'
# SERIES = 'causal_impact_4'
SERIES='mlr_2'

MODEL_VERSION='v1'
XAI_FLAG="TRUE"

BQ_PROJECT = PROJECT_ID
BQ_DATASET = SERIES.replace('-','_')
BQ_TABLE = EXPERIMENT

BQ_SOURCE1 = 'bigquery-public-data.new_york.citibike_trips'
BQ_SOURCE2 = 'bigquery-public-data.new_york.citibike_stations'
BQ_TABLE_COPY="hybrid-vertex.causal_impact_4.control_group1_grp_a"

viz_limit = 12

EXPERIMENT_NAME = f"nyc_{BQ_DATASET}_{MODEL_VERSION}"
print(f'EXPERIMENT_NAME: {EXPERIMENT_NAME}')

EXPERIMENT_NAME: nyc_mlr_2_v1


In [3]:
from google.cloud import bigquery

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

from google.cloud import aiplatform as vertex_ai

bq = bigquery.Client(project=PROJECT_ID)

vertex_ai.init(
    project=PROJECT_ID, 
    location=REGION,
    # credentials=credentials
)

In [4]:
# CUSTOMIZE
TARGET_COLUMN = 'num_trips'
TIME_COLUMN = 'starttime'
SERIES_COLUMN = 'start_station_name'
COVARIATE_COLUMNS = [
    'avg_tripduration', 
    'pct_subscriber', 
    'ratio_gender', 
    'capacity'
] # could be empty

BQ_TABLE_GROUP_A="control_group1_grp_a"
# BQ_TABLE_GROUP_B="control_group1_grp_b"

VERTEX_AI_MODEL_ID='v1_mlr_a'

## New BQ dataset

In [5]:
ds = bigquery.Dataset(f"{PROJECT_ID}.{BQ_DATASET}")
ds.location = 'us' #REGION
ds.labels = {'notebook': f"{EXPERIMENT}"}
ds = bq.create_dataset(dataset = ds, exists_ok = True)

print(f" ds.dataset_id: {ds.dataset_id}")
print(f" ds.full_dataset_id: {ds.full_dataset_id}")

 ds.dataset_id: mlr_2
 ds.full_dataset_id: hybrid-vertex:mlr_2


In [6]:
query = f"""
    CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}` AS (
        SELECT 
            * 
        FROM `{BQ_TABLE_COPY}`
    );
"""
print(query)
# `hybrid-vertex.causal_impact_4.control_group1_grp_a`


    CREATE OR REPLACE TABLE `hybrid-vertex.mlr_2.control_group1_grp_a` AS (
        SELECT 
            * 
        FROM `hybrid-vertex.causal_impact_4.control_group1_grp_a`
    );



In [7]:
job = bq.query(query)
job.result()
print(job.state, (job.ended-job.started).total_seconds())

DONE 1.329


## Key Dates

In [8]:
query = f"""
    WITH
        SPLIT AS (
            SELECT splits, min({TIME_COLUMN}) as mindate, max({TIME_COLUMN}) as maxdate
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}`
            GROUP BY splits
        ),
        TRAIN AS (
            SELECT mindate as start_date
            FROM SPLIT
            WHERE splits ='TRAIN'
        ),
        VAL AS (
            SELECT mindate as val_start
            FROM SPLIT
            WHERE splits = 'VALIDATE'
        ),
        TEST AS (
            SELECT mindate as test_start, maxdate as end_date
            FROM SPLIT
            WHERE splits = 'TEST'
        )
    SELECT * EXCEPT(pos) FROM
    (SELECT *, ROW_NUMBER() OVER() pos FROM TRAIN)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM VAL)
    USING (pos)
    JOIN (SELECT *, ROW_NUMBER() OVER() pos FROM TEST)
    USING (pos)
"""
keyDates = bq.query(query).to_dataframe()
keyDates

,start_date,val_start,test_start,end_date
0,2013-07-01,2016-05-14,2016-07-23,2016-09-30


In [9]:
query = f"""
    SELECT {SERIES_COLUMN}, {TIME_COLUMN}, {TARGET_COLUMN}, splits,
        {', '.join(COVARIATE_COLUMNS)}
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}`
    ORDER by {SERIES_COLUMN}, {TIME_COLUMN}
"""
rawSeries = bq.query(query).to_dataframe()

## Train MLR - Group A

In [10]:
# CUSTOMIZE
forecast_granularity = 'DAY'
forecast_horizon = 7 #14
forecast_test_length = 14
#forecast_val_length = 14

## what's a bug look like?

In [11]:
query = f"""
    CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`
    OPTIONS
      (
        model_type = 'linear_reg',
        input_label_cols = ['{TARGET_COLUMN}'],
        model_registry="vertex_ai", 
        vertex_ai_model_id='{VERTEX_AI_MODEL_ID}',
        vertex_ai_model_version_aliases=['{MODEL_VERSION}', 'experimental'],
        enable_global_explain={XAI_FLAG}
      ) AS
    SELECT {TIME_COLUMN}, {TARGET_COLUMN},
        {', '.join(COVARIATE_COLUMNS)}
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}`
    WHERE splits in ('TRAIN','VALIDATE')
"""
print(query)


    CREATE OR REPLACE MODEL `hybrid-vertex.mlr_2.control_group1_grp_a_mlr_v1`
    OPTIONS
      (
        model_type = 'linear_reg',
        input_label_cols = ['num_trips'],
        model_registry="vertex_ai", 
        vertex_ai_model_id='v1_mlr_a',
        vertex_ai_model_version_aliases=['v1', 'experimental'],
        enable_global_explain=TRUE
      ) AS
    SELECT starttime, num_trips,
        avg_tripduration, pct_subscriber, ratio_gender, capacity
    FROM `hybrid-vertex.mlr_2.control_group1_grp_a`
    WHERE splits in ('TRAIN','VALIDATE')



In [12]:
job = bq.query(query)
job.result()
print(job.state, (job.ended-job.started).total_seconds())

DONE 36.533


### Review Input Features

In [13]:
query = f"""
    SELECT *
    FROM ML.FEATURE_INFO(MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`)
"""
featureInfo = bq.query(query).to_dataframe()
featureInfo.head()

,input,min,max,mean,median,stddev,category_count,null_count,dimension
0,starttime,NaN,NaN,NaN,NaN,NaN,1089,0,<NA>
1,avg_tripduration,116.666667,45776.5,817.984338,680.78125,1404.576554,<NA>,0,<NA>
2,pct_subscriber,0.000000,1.0,0.921349,0.96000,0.110237,<NA>,0,<NA>
3,ratio_gender,0.000000,20.0,2.478870,2.00000,2.000844,<NA>,0,<NA>
4,capacity,0.000000,91.0,40.583178,39.00000,26.488621,<NA>,0,<NA>


In [14]:
query = f"""
    SELECT *
    FROM ML.TRAINING_INFO(MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`)
"""
trainingInfo = bq.query(query).to_dataframe()
trainingInfo.head()

,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,5,3101.960828,6550.015469,0.8,1872
1,0,4,3123.625647,6601.246628,0.8,2016
2,0,3,3185.819015,6727.225416,0.8,2130
3,0,2,3382.700092,6983.005701,0.8,1951
4,0,1,4482.891787,7293.532606,0.4,2005


## Forecast Evaluation

In [15]:
query = f"""
    SELECT *
    FROM ML.EVALUATE(
        MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`,
        (
            SELECT {TIME_COLUMN}, {TARGET_COLUMN},
                {', '.join(COVARIATE_COLUMNS)}
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}`
            WHERE splits = 'TEST'
        )
    )
"""
metrics = bq.query(query).to_dataframe()
metrics

,mean_absolute_error,mean_squared_error,mean_squared_log_error,median_absolute_error,r2_score,explained_variance
0,45.043066,3550.440725,4.042181,31.721676,-0.587681,-0.074246


## Explainability

* if `XAI_FLAG` set to `TRUE`

### local XAI

In [16]:
query = f"""
    CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_xai_{MODEL_VERSION}` AS (
    SELECT
      *
    FROM
      ML.EXPLAIN_PREDICT(
          MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`,
          (
            SELECT 
                {TIME_COLUMN}, 
                {TARGET_COLUMN},
                {', '.join(COVARIATE_COLUMNS)},
                {SERIES_COLUMN}
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}` 
            WHERE splits = 'TEST'
          ),
          STRUCT(3 as top_k_features)
        )
    );
"""
print(query)
# overview = bq.query(query).to_dataframe()
# overview.head(2)


    CREATE OR REPLACE TABLE `hybrid-vertex.mlr_2.control_group1_grp_a_xai_v1` AS (
    SELECT
      *
    FROM
      ML.EXPLAIN_PREDICT(
          MODEL `hybrid-vertex.mlr_2.control_group1_grp_a_mlr_v1`,
          (
            SELECT 
                starttime, 
                num_trips,
                avg_tripduration, pct_subscriber, ratio_gender, capacity,
                start_station_name
            FROM `hybrid-vertex.mlr_2.control_group1_grp_a` 
            WHERE splits = 'TEST'
          ),
          STRUCT(3 as top_k_features)
        )
    );



In [17]:
job = bq.query(query)
job.result()
print(job.state, (job.ended-job.started).total_seconds())

DONE 1.43


### global XAI

In [18]:
query = f"""
    SELECT
      *
    FROM
      ML.GLOBAL_EXPLAIN(
          MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`
    )
"""
# print(query)
overview = bq.query(query).to_dataframe()
overview

,feature,attribution
0,capacity,53.368157
1,starttime,40.251806
2,ratio_gender,2.727573
3,pct_subscriber,0.282598
4,avg_tripduration,0.166120


In [19]:
# job = bq.query(query)
# job.result()
# print(job.state, (job.ended-job.started).total_seconds())

## Forecast Test Set

In [20]:
query = f"""
    SELECT *
    FROM ML.PREDICT(
        MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`,
        (
            SELECT 
                {TIME_COLUMN}, 
                {TARGET_COLUMN},
                {', '.join(COVARIATE_COLUMNS)},
                {SERIES_COLUMN}
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}`
        )
        )
"""
forecast = bq.query(query).to_dataframe()
forecast
# print(query)

,predicted_num_trips,starttime,num_trips,avg_tripduration,pct_subscriber,ratio_gender,capacity,start_station_name
0,34.604236,2016-07-08,216,749.777778,0.916667,1.769231,0,Lafayette St & Jersey St N
1,38.486356,2016-07-09,221,857.588235,0.859729,1.695122,0,Lafayette St & Jersey St N
2,60.794206,2016-07-12,298,834.882550,0.929530,2.725000,0,Lafayette St & Jersey St N
3,-11.077209,2016-07-19,89,751.865169,0.966292,1.870968,0,Lafayette St & Jersey St N
4,194.788827,2016-07-17,201,1512.870647,0.855721,1.481481,0,Lafayette St & Jersey St N
...,...,...,...,...,...,...,...,...
4254,33.748675,2016-05-23,43,950.511628,0.906977,1.866667,0,Lafayette Ave & Fort Greene Pl
4255,-36.459672,2016-06-21,43,671.697674,0.976744,2.307692,0,Lafayette Ave & Fort Greene Pl
4256,21.582573,2016-07-11,43,1156.209302,0.837209,1.047619,0,Lafayette Ave & Fort Greene Pl
4257,-36.909727,2013-07-26,43,752.720930,0.837209,1.687500,0,Lafayette Ave & Fort Greene Pl


In [21]:
# CUSTOMIZE
query = f"""
CREATE OR REPLACE TABLE `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_pred_Test` AS (
    SELECT * FROM ML.PREDICT(
            MODEL `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}_mlr_{MODEL_VERSION}`,
            (
                SELECT
                {TIME_COLUMN}, 
                {TARGET_COLUMN},
                {', '.join(COVARIATE_COLUMNS)},
                {SERIES_COLUMN}
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE_GROUP_A}`
                WHERE splits = 'TEST'
            )
            )
)
"""
job = bq.query(query = query)
job.result()
(job.ended-job.started).total_seconds()

1.302

# Register Model in Vertex AI

In [22]:
# Initiate Vertex AI Model Registry for `VERTEX_AI_MODEL_ID` model entry
registry = vertex_ai.models.ModelRegistry(VERTEX_AI_MODEL_ID)

In [23]:
# Get model versions
versions = registry.list_versions()

for version in versions:
    version_id = version.version_id
    version_created_time = datetime.fromtimestamp(
        version.version_create_time.timestamp()
    ).strftime("%m/%d/%Y %H:%M:%S")
    version_aliases = version.version_aliases
    print(
        f"Model version {version_id} was created at {version_created_time} with aliases {version_aliases}",
    )

Getting versions for projects/hybrid-vertex/locations/us-central1/models/v1_mlr_a
Model version 1 was created at 05/04/2023 12:35:12 with aliases ['default']
Model version 2 was created at 05/04/2023 13:18:24 with aliases ['v2']
Model version 3 was created at 05/04/2023 14:43:54 with aliases ['v3']
Model version 4 was created at 05/14/2023 23:33:12 with aliases ['v1', 'experimental']


In [21]:
# # Get the model
# model = registry.get_model(version="1")
# print(model)

resource name: projects/934903580331/locations/us-central1/models/v1_mlr_a


In [22]:
# # Get the model
# model = registry.get_model(version="v2")
# print(model)

resource name: projects/934903580331/locations/us-central1/models/v1_mlr_a


In [24]:
# Get the model
model = registry.get_model(version="v1")
print(model)

resource name: projects/934903580331/locations/us-central1/models/v1_mlr_a


# TODO: Model Evlauation Pipeline

In [53]:
# kfp
import kfp
import kfp.v2.dsl
from kfp.v2.google import client as pipelines_client
from kfp.v2.dsl import (Artifact, Dataset, Input, InputPath, Model, Output,
                        OutputPath, component)

import google.cloud.aiplatform as vertex_ai

vertex_ai.init(
    project=PROJECT_ID,
    location=REGION
)

In [54]:
PIPELINE_DISPLAY_NAME = f"eval-pipe-{MODEL_VERSION}"  # @param {type:"string"}
print(PIPELINE_DISPLAY_NAME)

eval-pipe-v1


In [55]:
@kfp.dsl.pipeline(name=PIPELINE_DISPLAY_NAME)
def evaluation_custom_tabular_feature_attribution_pipeline(
    project: str,
    location: str,
    root_dir: str,
    model_name: str,
    bq_dataset_name: str,
    target_field_name: str,
    # batch_predict_gcs_source_uris: list,
    eval_bq_processed: str,
    batch_predict_instances_format: str,
    batch_predict_sample_size: int,
    batch_predict_predictions_format: str = "bigquery",
    batch_predict_machine_type: str = "n1-standard-4",
):

    from google_cloud_pipeline_components.aiplatform import ModelBatchPredictOp
    from google_cloud_pipeline_components.experimental.evaluation import (
        EvaluationDataSamplerOp, GetVertexModelOp,
        ModelEvaluationFeatureAttributionOp, ModelEvaluationRegressionOp,
        ModelImportEvaluationOp, TargetFieldDataRemoverOp)

    # Get the Vertex AI model resource
    get_model_task = GetVertexModelOp(model_resource_name=model_name)

    # Run Data-sampling task
    data_sampler_task = EvaluationDataSamplerOp(
        project=project,
        location=location,
        root_dir=root_dir,
        # gcs_source_uris=batch_predict_gcs_source_uris,
        bigquery_source_uri=eval_bq_processed,
        instances_format=batch_predict_instances_format,
        sample_size=batch_predict_sample_size,
    )

    # Run Data-splitter task
    data_splitter_task = TargetFieldDataRemoverOp(
        project=project,
        location=location,
        root_dir=root_dir,
        # gcs_source_uris=data_sampler_task.outputs["gcs_output_directory"],
        bigquery_source_uri=data_sampler_task.outputs["bigquery_output_table"],
        instances_format=batch_predict_instances_format,
        target_field_name=target_field_name,
    )

    # Run Batch Explanations
    batch_explain_task = ModelBatchPredictOp(
        project=project,
        location=location,
        model=get_model_task.outputs["model"],
        job_display_name="model-registry-batch-predict-evaluation",
        # gcs_source_uris=data_splitter_task.outputs["gcs_output_directory"],
        bigquery_source_input_uri=data_splitter_task.outputs["bigquery_output_table"],
        instances_format=batch_predict_instances_format,
        predictions_format=batch_predict_predictions_format,
        # gcs_destination_output_uri_prefix=root_dir,
        bigquery_destination_output_uri=f'bq://{project}.{bq_dataset_name}',
        machine_type=batch_predict_machine_type,
        # Set the explanation parameters
        generate_explanation=True,
    )

    # Run evaluation based on prediction type and feature attribution component.
    # After, import the model evaluations to the Vertex AI Model resource.
    eval_task = ModelEvaluationRegressionOp(
        project=project,
        location=location,
        root_dir=root_dir,
        # predictions_gcs_source=batch_explain_task.outputs["gcs_output_directory"],
        predictions_bigquery_source=batch_explain_task.outputs["bigquery_output_table"],
        # ground_truth_format="jsonl",
        ground_truth_format="bigquery",
        # ground_truth_gcs_source=data_sampler_task.outputs["gcs_output_directory"],
        ground_truth_bigquery_source=data_sampler_task.outputs["bigquery_output_table"],
        # predictions_format=batch_predict_predictions_format,
        predictions_format='bigquery', # batch_predict_predictions_format,
        # prediction_score_column="prediction",
        target_field_name=target_field_name,
        model=get_model_task.outputs["model"],
    )

    # # Get Feature Attributions
    # feature_attribution_task = ModelEvaluationFeatureAttributionOp(
    #     project=project,
    #     location=location,
    #     root_dir=root_dir,
    #     predictions_format="bigquery",
    #     # predictions_gcs_source=batch_explain_task.outputs["gcs_output_directory"],
    #     predictions_bigquery_source=batch_explain_task.outputs["bigquery_output_table"],
    # )

    ModelImportEvaluationOp(
        regression_metrics=eval_task.outputs["evaluation_metrics"],
        # feature_attributions=feature_attribution_task.outputs["feature_attributions"],
        model=get_model_task.outputs["model"],
        dataset_type=batch_predict_instances_format,
    )

In [56]:
kfp.v2.compiler.Compiler().compile(
    pipeline_func=evaluation_custom_tabular_feature_attribution_pipeline,
    package_path=f"{PIPELINE_DISPLAY_NAME}.json",
)

In [57]:
# BQ_DATASET
REGION = 'us-central1'
BUCKET_NAME = 'vertex-forecast-22'
GCS_BUCKET_URI =f'gs://{BUCKET_NAME}'

PIPELINE_ROOT = f"{GCS_BUCKET_URI}/pipeline_root/{BQ_DATASET}_{MODEL_VERSION}"
batch_predict_sample_size = 5
BQ_EVAL_TEST_SET='bq://hybrid-vertex.mlr_2.control_group1_grp_a_pred_Test'

SERVICE_ACCOUNT = 'notebooksa@hybrid-vertex.iam.gserviceaccount.com'

In [58]:

parameters = {
    "project": PROJECT_ID,
    "location": REGION,
    "root_dir": PIPELINE_ROOT,
    "bq_dataset_name":BQ_DATASET,
    "eval_bq_processed":BQ_EVAL_TEST_SET,
    "model_name": model.resource_name,
    "target_field_name": "num_trips",
    # "batch_predict_gcs_source_uris": [
    #     BUCKET_URI + "/" + "test_file_with_ground_truth.jsonl"
    # ],
    "batch_predict_instances_format": "bigquery",
    "batch_predict_sample_size": batch_predict_sample_size,
}
     

In [59]:
job = vertex_ai.PipelineJob(
    display_name=PIPELINE_DISPLAY_NAME,
    template_path=f"{PIPELINE_DISPLAY_NAME}.json",
    parameter_values=parameters,
    enable_caching=False,
)

job.run(service_account=SERVICE_ACCOUNT)

Creating PipelineJob
PipelineJob created. Resource name: projects/934903580331/locations/us-central1/pipelineJobs/eval-pipe-v1-20230515101352
To use this PipelineJob in another session:
pipeline_job = aiplatform.PipelineJob.get('projects/934903580331/locations/us-central1/pipelineJobs/eval-pipe-v1-20230515101352')
View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/eval-pipe-v1-20230515101352?project=934903580331
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/eval-pipe-v1-20230515101352 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/eval-pipe-v1-20230515101352 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/eval-pipe-v1-20230515101352 current state:
PipelineState.PIPELINE_STATE_RUNNING
PipelineJob projects/934903580331/locations/us-central1/pipelineJobs/eval-pipe-v1-2023051

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [model-evaluation-regression].; Job (project_id = hybrid-vertex, job_id = 1824527945864052736) is failed due to the above error.; Failed to handle the job: {project_number = 934903580331, job_id = 1824527945864052736}"
